In [25]:
# Library
import os
import random
import pickle
import gc
import warnings
import seaborn as sns
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
from datetime import datetime
from matplotlib import font_manager, rc
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.manifold import LocallyLinearEmbedding
from sklearn.cluster import KMeans, MiniBatchKMeans
# from category_encoders import TargetEncoder
from sklearn.preprocessing import (
    StandardScaler, PowerTransformer, OrdinalEncoder,
    OneHotEncoder, FunctionTransformer, PolynomialFeatures, LabelEncoder,
)
from sklearn.decomposition import PCA, IncrementalPCA, KernelPCA
from sklearn.feature_selection import SelectKBest, SelectPercentile, RFE
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import (
    LogisticRegression, LinearRegression, Ridge, Lasso,
    SGDRegressor, ElasticNet
)
from sklearn.model_selection import (
    train_test_split, cross_val_score, cross_validate,
    GridSearchCV, KFold, cross_val_predict
)
from sklearn.metrics import (
    roc_auc_score, mean_squared_error, make_scorer, accuracy_score, log_loss
)
from sklearn import set_config, datasets
from catboost import (
    CatBoostRegressor, CatBoostClassifier,
)
# import category_encoders as ce
# from sklearn.pipeline import (
#     Pipeline, FeatureUnion, make_pipeline
# )
from sklearn.ensemble import (
    RandomForestClassifier, StackingClassifier, StackingRegressor,
    GradientBoostingRegressor, VotingClassifier, VotingRegressor,
    HistGradientBoostingRegressor, GradientBoostingClassifier,
    BaggingClassifier, AdaBoostClassifier, RandomForestRegressor,ExtraTreesRegressor
)
from lightgbm import LGBMRegressor, LGBMClassifier
from sklearn.svm import SVC, SVR, LinearSVC
from xgboost import XGBRegressor, XGBClassifier
from catboost import CatBoostClassifier, Pool
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import make_moons

import xgboost as xgb
import lightgbm as lgb
import re
import math
import optuna

from scipy.stats import zscore

%matplotlib inline

warnings.filterwarnings("ignore")

In [26]:
df = pd.read_pickle('matches_df.pkl')

In [27]:
# train_sample = pd.read_csv('train_sample.csv')

In [28]:
df[1230:1240]

,index,date_GMT,home_team_name,away_team_name,home_team_corner_count,away_team_corner_count,home_team_shots,away_team_shots,home_team_shots_on_target,away_team_shots_on_target,...,away_team_goal_count,home_team_goal_count_half_time,away_team_goal_count_half_time,home_team_yellow_cards,home_team_red_cards,away_team_yellow_cards,away_team_red_cards,home_team_result,game_points,5_games_result
1230,175,2018-09-26 05:00:00,Gangwon,Suwon Bluewings,2,2,18,5,5,0,...,0,0,0,1,0,1,0,승,3,0.0
1231,187,2018-10-06 05:00:00,Gangwon,Pohang Steelers,4,4,14,13,7,5,...,1,1,0,2,0,0,0,무,0,3.0
1232,209,2018-11-04 07:00:00,Gangwon,Jeonnam Dragons,4,7,11,15,6,5,...,0,1,0,4,0,1,0,승,3,3.0
1233,212,2018-11-10 05:00:00,Gangwon,Incheon United,8,4,21,20,10,6,...,3,1,2,0,0,1,0,패,-3,0.0
1234,223,2018-12-01 05:00:00,Gangwon,Daegu,8,1,20,6,7,3,...,1,0,1,1,0,1,0,패,-3,0.0
1235,6,2018-03-04 05:00:00,Gyeongnam,Sangju Sangmu,4,10,7,14,4,7,...,1,1,0,1,1,1,0,승,3,0.0
1236,9,2018-03-10 05:00:00,Gyeongnam,Jeju United,7,2,23,10,10,2,...,0,1,0,4,0,3,1,승,3,0.0
1237,26,2018-04-07 07:00:00,Gyeongnam,Daegu,2,3,8,11,4,3,...,1,0,1,2,0,2,0,무,0,0.0
1238,32,2018-04-11 10:30:00,Gyeongnam,Jeonbuk Motors,5,4,20,22,4,8,...,4,0,3,0,0,1,0,패,-3,0.0
1239,48,2018-04-22 07:00:00,Gyeongnam,Ulsan,2,7,12,14,4,5,...,0,0,0,0,0,1,0,무,0,3.0


## Preprocessing

In [29]:
X = df
y = df['home_team_result']

#특정 인덱스 기준으로 데이터 분리
split_index = 1800

# train, test, y_train, y_test 데이터 분리
train = X.iloc[:split_index]
test = X.iloc[split_index:]
y_train = y.iloc[:split_index]
y_test = y.iloc[split_index:]


# train, test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42) ## 이 방법에 의문을 품는 바이요!!

In [30]:
def team_encoding(train):
    train['home_win'] = train['home_team_result'].apply(lambda x: 1 if x=='승' else 0) # home_win 열 추가, 승리인 경우 1, 아닌 경우 0
    dic = {}
    # 각 홈팀별 이긴 경기 수를 딕셔너리에 저장
    for team in train['home_team_name'].unique():
        value = train[train['home_team_name'] == team]['home_win'].sum() 
        #home_team_name  열에서 고유한 팀 이름을 가져와 각 팀이 홈에서 이긴 경기 수 계산, 이 값을 dic에 저장
        dic[team] = value

    label_dic={}
    # 승리 횧수를 기준으로 오름차순 정렬, 각 팀에 대해 라벨 부여, 승리 횟수가 적은 팀부터 0,1,2 의 라벨을 부여
    for idx, (team, _) in enumerate(sorted(dic.items(), key= lambda x: x[1])):
        label_dic[team] = idx
    
    return label_dic


''' 홈팀 득점 이동평균 계산 함수 '''

def homeGoal_day_mean(train, test, day):
    train[f'home_Goal_{day}_mean'] = -1  # 초기값 -1로 설정
    test[f'home_Goal_{day}_mean'] = -1
    
    teams = train['home_team_name'].unique()
    for team in tqdm(teams): # train에서 고유 팀 이름을 가져오고 이를 시각적으로 표시해줌 : tqdm
        team_df = train[train['home_team_name'] == team]
        # 롤링 윈도우 크기 설정
        ch_day = len(team_df) if len(team_df) < day else day # 팀의 경기 수가 주어진 day 보다 적으면, 경기 수 만큼의 윈도우 크기 사용
        idx = team_df['home_team_goal_count'].rolling(ch_day).mean().index.values # 롤링 윈도우 평균 계산
        val = team_df['home_team_goal_count'].rolling(ch_day).mean().values
        train[f'home_Goal_{day}_mean'].loc[idx] = val
        test_idx = test[test['home_team_name'] == team].index
        test[f'home_Goal_{day}_mean'].loc[test_idx] = val[-1]
    # 결측값 처리
    train[f'home_Goal_{day}_mean'] = train[f'home_Goal_{day}_mean'].fillna(0)

''' 원정팀 득점 이동평균 계산 함수 '''
def awayGoal_day_mean(train, test, day):
    # 초기값 설정
    train[f'away_Goal_{day}_mean'] = -1
    test[f'away_Goal_{day}_mean'] = -1
    
    teams = train['away_team_name'].unique()
    for team in tqdm(teams):
        team_df = train[train['away_team_name'] == team]
        # 롤링 윈도우 크기 설정
        ch_day = len(team_df) if len(team_df) < day else day
        idx = team_df['away_team_goal_count'].rolling(ch_day).mean().index.values
        val = team_df['away_team_goal_count'].rolling(ch_day).mean().values
        train[f'away_Goal_{day}_mean'].loc[idx] = val
        test_idx = test[test['away_team_name'] == team].index
        test[f'away_Goal_{day}_mean'].loc[test_idx] = val[-1]
    # 결측값 처리
    train[f'away_Goal_{day}_mean'] = train[f'away_Goal_{day}_mean'].fillna(0)


# 홈팀 승리율 평균 계산 함수
def homeWin_day_mean(train, test, day):
    # 초기값 설정
    train[f'home_winRate_{day}_mean'] = -1
    test[f'home_winRate_{day}_mean'] = -1
    train['win'] = train['home_team_result'].apply(lambda x: 1 if x == '승' else 0)
    
    # 각 팀별로 진행
    teams = train['home_team_name'].unique()
    for team in tqdm(teams):
        team_df = train[train['home_team_name'] == team]
        # 롤링 윈도우 크기 설정
        ch_day = len(team_df) if len(team_df) < day else day
        idx = team_df['win'].rolling(ch_day).mean().index.values
        val = team_df['win'].rolling(ch_day).mean().values
        train[f'home_winRate_{day}_mean'].loc[idx] = val
        test_idx = test[test['home_team_name'] == team].index
        test[f'home_winRate_{day}_mean'].loc[test_idx] = val[-1]
    # 임시 컬럼 제거
    train.drop(columns=['win'], inplace=True)
    # 결측값 처리
    train[f'home_winRate_{day}_mean'] = train[f'home_winRate_{day}_mean'].fillna(0)

# 원정팀 승리율 평균 계산 함수
def awayWin_day_mean(train, test, day):
    # 초기값 설정
    train[f'away_winRate_{day}_mean'] = -1
    test[f'away_winRate_{day}_mean'] = -1
    train['win'] = train['home_team_result'].apply(lambda x: 1 if x == '패' else 0)
    
    # 각 팀별로 진행
    teams = train['away_team_name'].unique()
    for team in tqdm(teams):
        team_df = train[train['away_team_name'] == team]
        # 롤링 윈도우 크기 설정
        ch_day = len(team_df) if len(team_df) < day else day
        idx = team_df['win'].rolling(ch_day).mean().index.values
        val = team_df['win'].rolling(ch_day).mean().values
        train[f'away_winRate_{day}_mean'].loc[idx] = val
        test_idx = test[test['away_team_name'] == team].index
        test[f'away_winRate_{day}_mean'].loc[test_idx] = val[-1]
    # 임시 컬럼 제거
    train.drop(columns=['win'], inplace=True)
    # 결측값 처리
    train[f'away_winRate_{day}_mean'] = train[f'away_winRate_{day}_mean'].fillna(0)

# 홈팀 평균 계산 함수
def home_day_mean(train, test, columns, day):
    for column in tqdm(columns):
        teams = train['home_team_name'].values
        train[f'home_{column}_{day}_mean'] = -1
        test[f'home_{column}_{day}_mean'] = -1

        for team in tqdm(teams):
            team_df = train[train['home_team_name'] == team]
            idx = team_df[column].rolling(day).mean().index.values
            val = team_df[column].rolling(day).mean().values
            train[f'home_{column}_{day}_mean'].loc[idx] = val
            test_idx = test[test['home_team_name'] == team].index
            test[f'home_{column}_{day}_mean'].loc[test_idx] = val[-1]
        # 결측값 처리
        train[f'home_{column}_{day}_mean'] = train[f'home_{column}_{day}_mean'].fillna(0)
        test[f'home_{column}_{day}_mean'] = test[f'home_{column}_{day}_mean'].fillna(0)

# 원정팀 평균 계산 함수
def away_day_mean(train, test, columns, day):
    for column in tqdm(columns):
        teams = train['away_team_name'].values
        train[f'away_{column}_{day}_mean'] = -1
        test[f'away_{column}_{day}_mean'] = -1

        for team in tqdm(teams):
            team_df = train[train['away_team_name'] == team]
            idx = team_df[column].rolling(day).mean().index.values
            val = team_df[column].rolling(day).mean().values
            train[f'away_{column}_{day}_mean'].loc[idx] = val
            test_idx = test[test['away_team_name'] == team].index
            test[f'away_{column}_{day}_mean'].loc[test_idx] = val[-1]
        # 결측값 처리
        train[f'away_{column}_{day}_mean'] = train[f'away_{column}_{day}_mean'].fillna(0)
        test[f'away_{column}_{day}_mean'] = test[f'away_{column}_{day}_mean'].fillna(0)

# 전처리 함수
def preprocessing(train, test, dic):
    # 년과 월일로 나누기
    train['date_GMT'] = train['date_GMT'].dt.strftime('%Y%m%d')
    test['date_GMT'] = test['date_GMT'].dt.strftime('%Y%m%d')
    train['year'] = train['date_GMT'].apply(lambda x : int(x[0:4]))
    train['month'] = train['date_GMT'].apply(lambda x : int(x[5:7]))
    # train['day'] = train['date_GMT'].apply(lambda x : int(x[8:10]))

    test['year'] = test['date_GMT'].apply(lambda x : int(x[0:4]))
    test['month'] = test['date_GMT'].apply(lambda x : int(x[5:7]))
    # test['day'] = test['date_GMT'].apply(lambda x : int(x[8:10]))
    train.drop(columns=['date_GMT'], inplace=True)
    test.drop(columns=['date_GMT'], inplace=True)

    # 팀 인코딩 적용
    label_dic = dic
    train['home_team_name'] = train['home_team_name'].apply(lambda x: label_dic[x])
    train['away_team_name'] = train['away_team_name'].apply(lambda x: label_dic[x])
    test['home_team_name'] = test['home_team_name'].apply(lambda x: label_dic[x])
    test['away_team_name'] = test['away_team_name'].apply(lambda x: label_dic[x])

    # 5일간 홈팀 승리 비율 계산
    homeWin_day_mean(train, test, 5)
    # 5일간 원정팀 승리 비율 계산
    awayWin_day_mean(train, test, 5)

    # 5일간 홈팀 평균 득점 계산
    homeGoal_day_mean(train, test, 5)
    # 5일간 원정팀 평균 득점 계산
    awayGoal_day_mean(train, test, 5)

    # 불필요한 컬럼 제거
    train = train.drop(columns=['home_win', 'index','home_team_goal_count','away_team_goal_count','home_team_result','game_points'])
    test = test.drop(columns=['index','home_team_goal_count','away_team_goal_count','home_team_result','game_points'])

    return train, test


## Seed

In [31]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss

import warnings
warnings.filterwarnings("ignore")

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42)

## Train / Test

In [32]:
hometeam_list = list(train['home_team_name'].unique())
dic = team_encoding(train)
train, test= preprocessing(train, test, dic)
test_idx = test.index.values

# name encoding
cat = ['home_team_name','away_team_name']


# 승무패 인코딩
lec = LabelEncoder()
lec.fit(df['home_team_result'])
y_train = lec.transform(y_train)
y_test = lec.transform(y_test)

  0%|          | 0/17 [00:00<?, ?it/s]

100%|██████████| 17/17 [00:00<00:00, 684.75it/s]


## Feature scaler

In [33]:
from sklearn.preprocessing import MinMaxScaler

num_features = list(set(train.columns) - set(cat))
scaler = MinMaxScaler()
train[num_features] = scaler.fit_transform(train[num_features])
test[num_features] = scaler.fit_transform(test[num_features])

## model train

In [38]:
# 로지스틱 회귀 모델 생성 및 학습
model = RandomForestClassifier()
model.fit(train, y_train)

# 예측
y_pred = model.predict(test)

## Randomforest feature seletion

In [ ]:
# 피처 중요도 계산
importances = model.feature_importances_
feature_names = train.columns

# 피처 중요도를 기준으로 정렬하여 상위 10개 피처 선택
indices = np.argsort(importances)[::-1]
top_10_indices = indices[:]
top_10_features = feature_names[top_10_indices]

# 결과 출력
print('랜덤 포레스트로 선택된 상위 10개 피처:')
for i, feature in enumerate(top_10_features):
    print(f"{i+1}. {feature} (중요도: {importances[top_10_indices[i]]})")

## 상관계수 계산

In [42]:
# 상관계수 계산
corr_matrix = train.corr().abs()
# 상위 삼각 행렬
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
# 상관계수가 0.9 이상인 특징 찾기
to_drop = [column for column in upper.columns if any(upper[column] > 0.9)]
to_drop
# # 특징 제거
# df_reduced = df.drop(columns=to_drop)  
'''이거 할 필요가 있을 듯'''

['away_team_possession', 'home_team_free_kick', 'away_team_free_kick']

## L1 규제(Lasso)

In [43]:
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV

# alpha 값 후보군 설정
alpha_values = [0.001, 0.01, 0.1, 1, 10, 100]
param_grid = {'alpha': alpha_values}

# Lasso 모델과 GridSearchCV 설정
lasso = Lasso()
grid_search = GridSearchCV(lasso, param_grid, cv=5, scoring='neg_mean_squared_error')

# 최적의 alpha 값 찾기
grid_search.fit(train, y_train)
best_alpha = grid_search.best_params_['alpha']

In [ ]:
lasso = Lasso(alpha=0.1)  # alpha 값 조정 필요
lasso.fit(train, y_train)

# 가중치가 0이 아닌 특징 선택
selected_features = train.columns[lasso.coef_ != 0]
train_selected = train[selected_features]

In [41]:
# 정확도 계산
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.5147058823529411


In [45]:
model = lgb.LGBMClassifier(n_estimators=100, random_state=42)
model.fit(train, y_train) 
y_pred = model.predict(test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000077 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 526
[LightGBM] [Info] Number of data points in the train set: 1800, number of used features: 29
[LightGBM] [Info] Start training from score -1.243638
[LightGBM] [Info] Start training from score -0.933098
[LightGBM] [Info] Start training from score -1.144656
Accuracy: 0.5308823529411765


# 2013~2023  
### 많긴 하지만 정확도 떨어질 것으로 예상됨

# 2020~2023
### 데이터 수가 급격히 줄어듬